<img src="https://raw.githubusercontent.com/euroargodev/argopy/master/docs/_static/argopy_logo_long.png" alt="argopy logo" width="200"/> 

# Download and visualise core-Argo data over an oceanic region

This notebook shows how to:
- download Argo core data (i.e. temperature/salinity) for a space/time domain 
- manipulate the data (interpolation on standard depth levels)
- complement the dataset with the EOS10 variables
- run per-profile diagnostics, such as the mixed layer depth
- plot a map with floats trajectory
- plot core data
- plot complemented data

## Import and set-up

In [ ]:
from argopy import DataFetcher  # This is the class to load Argo data
from argopy.plot import scatter_map, scatter_plot # Visualisation methods

# Other usefull imports
import numpy as np
from matplotlib import pyplot as plt
import xarray as xr
xr.set_options(display_expand_attrs = False)

## Define an ocean region to work with

For this tutorial notebook, we'll work with Argo data from the core of the North Pacific Subtropical Gyre, south of the Kuroshio.

For your own use, you can simply overwrite the ``BOX`` variable content with your region.

Don't forget that if your selection is too large, data fetching may fail (although you could check with the ``parallel=True, progress=True`` arguments if it solve the problem).

In [ ]:
# Expected box format: [lon_min, lon_max, lat_min, lat_max, pres_min, pres_max, datim_min, datim_max]

BOX = [146, 152, 30, 34, 0, 2000, '2022-01', '2025-01']

## Load all data from this region

Note that if the BOX has a lot of profiles, the request can take up to a few minutes.  

In [ ]:
%%time
f = DataFetcher()  # Instantiate a data fetcher (here using all default options)
f = f.region(BOX)  # Define a data selection
f = f.load()       # Load data
f

In [ ]:
# Check the data structure (following xarray dataset model, very similar to a netcdf in memory):
ds = f.data
ds

**Exercise**:

Look at the xarray dataset global attribute ``Processing_history`` to get a description of what argopy has done with the raw data:

In [ ]:
ds.attrs['

You can check the data size through the argo xarray accessor:

In [ ]:
ds.argo

and check the corresponding simplified index of profiles (as a pandas.dataframe):

In [ ]:
df = f.index
df

## Data manipulation

**Exercise**:

Convert the collection of points into a collection of profiles 
with the `point2profile` method of the `argo` accessor

In [ ]:
%%time
dsp = ds.
dsp

**Exercise**:

Interpolate measurements along standard pressure levels, so that all profiles have a similar number of vertical levels

In [ ]:
b = np.arange(0., 2000., 5.0)  # standard pressure levels, in db
dsbp = dsp.argo.
dsbp

## Data completion

#### Use argopy methods

**Exercise**:

Compute the N2 stratification for all profiles

In [ ]:
dsp.argo.teos10(
dsp['N2']

#### Use your own diagnostic

**Exercise**:

Compute the mixed layer depth for each profiles

We can use the Boyer Montégut method based on the threshold of σ(10m) + 0.03 kg.m-3

In [ ]:
# This is the function to compute the mixed layder depth from a single profile:
def diag_mld(pres, dens0, dens0_ref_depth=10., theshold=0.03):
    idx = ~np.logical_or(np.isnan(pres), np.isnan(dens0))
    pres = pres[idx]
    dens0 = dens0[idx]
    if not any(pres < dens0_ref_depth) or all(pres < dens0_ref_depth):
        return np.nan
    else:
        index_ref_depth = np.argmin(np.abs(pres - dens0_ref_depth))
        density_at_ref_depth = dens0[index_ref_depth]
        index = np.min(
            np.where(dens0[index_ref_depth:] > density_at_ref_depth + theshold) + index_ref_depth
        )
        MLD = pres[index]
        return MLD

**Exercise**:
  
Before applying the diagnostic, we need to compute potential density:

In [ ]:
ds.argo.teos10(

**Exercice**:

Apply the reducing function on all profiles of the dataset

In [ ]:
# Don't forget to transform the collection of points into a collection of profiles:
dsp = ds.argo.point2profile()

In [ ]:
dsp['MLD'] = dsp.argo.reduce_profile(

## Data visualisation

### Make a map with all floats trajectory

In [ ]:
# We can directly use the index retrieved with the DataFetcher:
scatter_map(f.index, set_global=False, legend=False);

In [ ]:
# Customize the map:
fig, ax = scatter_map(f.index,
                      markersize=24,
                      markeredgecolor='w',
                      traj_color='gray',
                      legend=False)

### Plot time series

We will be using the argopy scatter_plot method that has the following signature:

```
scatter_plot(
    ds: xarray.core.dataset.Dataset,
    this_param,
    x='TIME',
    y='PRES',
    figsize=(18, 6),
    cmap=None,
    vmin=None,
    vmax=None,
    s=4,
)
```

In [ ]:
fig, ax, m = scatter_plot(ds, 'TEMP', vmin=2, vmax=20)
ax.set_title("%s ('%s' mission)" % ('TEMP', f.mission), fontdict={'weight': 'bold', 'size': 14});

In [ ]:
fig, ax, m = scatter_plot(ds, 'PSAL', vmin=34.5, vmax=35)
ax.set_title("%s ('%s' mission)" % ('PSAL', f.mission), fontdict={'weight': 'bold', 'size': 14});

**Exercise**:

Use the ``scatter_plot`` method and highlight the [17-19] degC layer from the interpolated data on standard pressure levels.

Superimpose the mixed layer depth with a simple ``plt.plot()``

In [ ]:
fig, ax, m = scatter_plot(dsbp, 'TEMP', y=
ax.plot(
ax.set_ylim(500, 0)

### Plot some variable against another

In [ ]:
scatter_plot(ds, 'N2', x = 'PSAL', y = 'TEMP', vmin=0, vmax=3e-5, figsize=(5,5), cbar=True)

# Load data using alternative user modes

In [ ]:
%%time
f_research = DataFetcher(mode='research')
f_research = f_research.region(BOX).load()
f_research

In [ ]:
f_research.data.attrs['Processing_history'].split(";")

In [ ]:
# to be compared with the less severe processing using the "standard" user mode:
f.data.attrs['Processing_history'].split(";")